In [1]:
import queue
import random
import math
import copy

In [2]:
class Board:
    def generateMatrix(self, N, rows, columns):
        """ Generate a matrix of sqrt(N+1) x sqrt(N+1) dimensions 
        Created by: Pedro
        """
        board = []
        count = 1
        for i in range(1, rows+1):
            row = []
            for j in range(1, columns + 1):
                row.append(count)
                count += 1
            board.append(row)
        board[rows-1][columns-1] = '*'
        return board

    def __init__(self, N):
        self.N = N
        self.rows = self.columns = int(math.sqrt(N+1))
        self.board = self.generateMatrix(self.N, self.rows, self.columns)
        self.goal = []
        for i in self.board:
            self.goal.append(tuple(i))
        self.goal = tuple(self.goal)
        self.empty = [self.rows - 1, self.columns - 1]

    def __repr__(self):
        string = ''
        for row in self.board:
            for num in row:
                if len(str(num)) == 1:
                    string += '   ' + str(num)
                elif len(str(num)) > 1:
                    string += '  ' + str(num)
            string += '\n'
        return string

    def convert_to_tuple(self, ar):
        result = []
        for i in ar:
            result.append(tuple(i))
        return tuple(result)

    def convert_to_list(self, tup):
        result = []
        for i in tup:
            result.append(list(i))
        return result

    def match(self, copy):
        a = Board(self.N)
        a.board = copy 
        for row in range(0, self.rows):
            for col in range(0, self.columns):
                if a.board[row][col] == '*':
                    a.empty = [row, col] 
        result = []
        for i in a.board:
            result.append(list(i))
        a.board = result 
        return a

    def move_up(self): # move empty block up
        try:
            if self.empty[0] != 0:
                tmp = self.board[self.empty[0]-1][self.empty[1]]
                self.board[self.empty[0]-1][self.empty[1]] = '*'
                self.board[self.empty[0]][self.empty[1]] = tmp
                self.empty = [self.empty[0]-1, self.empty[1]]
        except IndexError:
            pass

    def move_down(self): # move empty block down
        try:
            tmp = self.board[self.empty[0]+1][self.empty[1]]
            self.board[self.empty[0]+1][self.empty[1]] = '*'
            self.board[self.empty[0]][self.empty[1]] = tmp
            self.empty = [self.empty[0]+1, self.empty[1]]
        except IndexError:
            pass

    def move_right(self): # move empty block right
        try:
            tmp = self.board[self.empty[0]][self.empty[1]+1]
            self.board[self.empty[0]][self.empty[1]+1] = '*'
            self.board[self.empty[0]][self.empty[1]] = tmp
            self.empty = [self.empty[0], self.empty[1]+1]
        except IndexError:
            pass

    def move_left(self): # move empty block left
        try:
            if self.empty[1] != 0:
                tmp = self.board[self.empty[0]][self.empty[1]-1]
                self.board[self.empty[0]][self.empty[1]-1] = '*'
                self.board[self.empty[0]][self.empty[1]] = tmp
                self.empty = [self.empty[0], self.empty[1]-1]
        except IndexError:
            pass

    def shuffle(self, steps):
        for i in range(0, steps):
            direction = random.randrange(1, 5)
            if direction == 1:
                self.move_up()
            elif direction == 2:
                self.move_right()
            elif direction == 3:
                self.move_left()
            elif direction == 4:
                self.move_down()

    def solve_bfs(self):
        start = self.convert_to_tuple(self.board) 
        pred = {}
        visited = []
        frontier = queue.Queue()
        frontier.put(start)
        
        while frontier.qsize() > 0:
            tmp = frontier.get() 
            if tmp == self.goal:
                path = []
                while tmp != start:
                    path.append(pred[tmp][1])
                    tmp = pred[tmp][0]
                return path[::-1]
            
            if tmp not in visited:
                visited.append(tmp)
                tmpboard = self.match(tmp)  
                tmpboard.move_up()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'up']

                
                tmpboard = self.match(tmp)
                tmpboard.move_down()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'down']

                        
                tmpboard = self.match(tmp)
                tmpboard.move_right()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'right']

                
                tmpboard = self.match(tmp)
                tmpboard.move_left()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'left']

        raise Exception('There is no solution.')
    
    def solve_dfs(self):
        """ Modified the data structure used by the algorithm to be a stack
        Created by: Cassandra
        """
        start = self.convert_to_tuple(self.board) 
        pred = {}
        visited = []
        frontier = queue.LifoQueue() 
        frontier.put(start)
        
        while frontier.qsize() > 0:
            tmp = frontier.get() 
            if tmp == self.goal:
                path = []
                while tmp != start:
                    path.append(pred[tmp][1])
                    tmp = pred[tmp][0]
                return path[::-1]
            
            if tmp not in visited:
                visited.append(tmp)
                tmpboard = self.match(tmp) 
                tmpboard.move_up()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'up']

                
                tmpboard = self.match(tmp)
                tmpboard.move_down()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'down']

                        
                tmpboard = self.match(tmp)
                tmpboard.move_right()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'right']

                
                tmpboard = self.match(tmp)
                tmpboard.move_left()
                if self.convert_to_tuple(tmpboard.board) != tmp:
                    frontier.put(self.convert_to_tuple(tmpboard.board))
                    if self.convert_to_tuple(tmpboard.board) not in pred:
                        pred[self.convert_to_tuple(tmpboard.board)]=[tmp, 'left']

        raise Exception('There is no solution.')
    

In [3]:
print('BFS SOLUTION')
N = input("Please enter the size of the puzzle: ")
board = Board(int(N))
board.shuffle(20)
copyBoard = copy.deepcopy(board)
print(board)
path = board.solve_bfs()
print(path)

for dir in path:
    if dir == 'right':
        board.move_right()
    if dir == 'left':
        board.move_left()
    if dir == 'up':
        board.move_up()
    if dir == 'down':
        board.move_down()

print(board)

print('DFS SOLUTION')
print(copyBoard)
path = copyBoard.solve_dfs()
print(path)

for dir in path:
    if dir == 'right':
        copyBoard.move_right()
    if dir == 'left':
        copyBoard.move_left()
    if dir == 'up':
        copyBoard.move_up()
    if dir == 'down':
        copyBoard.move_down()

print(copyBoard)

BFS SOLUTION
Please enter the size of the puzzle: 8
   1   2   3
   5   8   7
   4   6   *

['up', 'left', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
   1   2   3
   4   5   6
   7   8   *

DFS SOLUTION
   1   2   3
   5   8   7
   4   6   *

['left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down']
   1   2   3
   4   5   6
   7   8   *

